In [8]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [9]:
pd.set_option('display.max_row', 10)
df = pd.read_csv('../website_classification.csv')
df.tail(10)

,website_url,cleaned_website_text,Category
923,https://adultcompass.com/avs/0/1/acw/index.html,crazy d sex worldthe best frames of the frank ...,Adult
924,http://www.tentacle-hentai.com/,tentacle hentai sex tentacle monsters fucking ...,Adult
925,https://www.hotsexsites.org/blog/,blog hot sex sitesblog how much are stripchat ...,Adult
926,http://www.allanimatedtgp.com/avs/cartoonposer,all animated tgpthis page doesnt existvideos c...,Adult
927,http://www.allanimatedtgp.com/tags,all animated tgp video categoriesall animated ...,Adult
928,http://www.allanimatedtgp.com/videos?t=9,amateur videos all animated tgpview over amate...,Adult
929,https://www.hentagon.com/,free hentai porn videos xxx anime sexlove hent...,Adult
930,https://www.banzaihentai.net/,banzaihentai discover new pleasuresdiscover ne...,Adult
931,https://www.fandomination.net/,fan domination sex cams shows bdsm girlfan do...,Adult
932,http://www.abcb.com/parents/legend.htm,the animé café a parents guide to animehelp ur...,Adult


In [10]:
computers = pd.read_csv('../Hierarchal model/Computer/computer_subcategory_classification.csv')
computers

,website_url,cleaned_website_text,Category
0,http://www.kasrl.org/jaffe.html,japanese female facial expression jaffe datase...,Computers and Technology
1,https://www.gmdhshell.com/,best predictive analytic software free academi...,Computers and Technology
2,http://en.wikipedia.org/wiki/Neural_network,neural network wikipedia neural network conten...,Computers and Technology
3,http://cadingandcoding.blogspot.com/,cading coding cading coding tuesday march auto...,Computers and Technology
4,http://www.panzercad.com/,welcome bluehost rate web host provider free c...,Computers and Technology
...,...,...,...
312,https://kryptomixer.io/,cryptomixer blender bitcoin mixer service cryp...,Cryptocurrency
313,https://www.walletexplorer.com/,walletexplorercom smart bitcoin block explorer...,Cryptocurrency
314,https://ultramixer.net/,ultramixer bitcoin mixer bitcoin tumbler best...,Cryptocurrency
315,https://nakedsecurity.sophos.com/2020/02/17/po...,police bust alleged operator of bitcoin mixing...,Cryptocurrency


In [11]:
urls = pd.read_csv("../URL Classification.csv",header=None)
urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562978 entries, 0 to 1562977
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1562978 non-null  int64 
 1   1       1562975 non-null  object
 2   2       1562978 non-null  object
dtypes: int64(1), object(2)
memory usage: 35.8+ MB


In [12]:
urls[2].value_counts()

Arts         253840
Society      243943
Business     240177
Computers    117962
Science      110286
              ...  
Games         56477
Kids          46182
Adult         35325
Home          28269
News           8989
Name: 2, Length: 15, dtype: int64

In [18]:
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os


class ScrapTool:   
    def visit_url(self,website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        #Set up Selenium webdriver
        PATH = 'C:/Users/User01/Downloads/chromedriver_win32/chromedriver.exe'

        # Set the Chrome webdriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Initialize the Chrome webdriver and pass the Chrome options
        service = Service(PATH)
        driver = webdriver.Chrome(service = service, options = chrome_options)

        #Load website
        driver.get(website_url)
        
        # Check if the cookie consent button is present
        cookie_button = None
        try:
            cookie_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "cookie-consent-button")))
        except:
            pass

        # Handle cookies if the button is present
        if cookie_button:
            cookie_button.click()

        # Extract HTML content
        html_content = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html_content,'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                    self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

In [14]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+',' ',text)
    return text

In [15]:
def content_generation(website):
    try:
        scrapTool = ScrapTool()
        web = scrapTool.visit_url(website)
        text = cleaning_text(web['website_text'])
        return text
    except Exception as e:
        print('Error: ',website)
        print(e)
        return None

In [21]:
websites = [
    "https://en.wikipedia.org/wiki/Carding_(fraud)#",
    "https://club2crd.cc/forumdisplay.php?f=54",
    "https://club2crd.cc/showthread.php?p=652434#post652434",
    "https://club2crd.cc/showthread.php?t=11791",
    "https://club2crd.cc/showthread.php?t=139049",
    "https://darkpro.net/index.php?ref=xranks",
    "https://darkpro.net/index.php?forums/carding-forum-trusted-and-verified-sellers.2/",
    "https://darkpro.net/index.php?forums/carding-forum-hackread.43/",
    "https://darkpro.net/index.php?forums/darkpro-cardable-websites.7/",
    "https://darkpro.net/index.php?forums/accounts-and-database-dumps.16/",
    "https://darkpro.net/index.php?threads/cardable-site-list-without-need-cvv2.29/",
    "https://darkpro.net/index.php?forums/darkpro-carding-tutorials.14/",
    "https://darkpro.net/index.php?threads/dl-ssn-passports-ticketing-cc-dumps-track-1-2-pin-100-paypal-100-fresh-base-2022-dumps-pin-tracks-1-2.4187/",
    "https://darkwebmafias.com/forums/cardable-website.24/",
    "https://darkwebmafias.com/forums/carding-tutorials.27/",
    "https://darkwebmafias.com/threads/carding-paypal-with-cvv.15/",
    "https://darkwebmafias.com/forums/online-carding.22/",
    "https://shadowcarders.com/",
    "https://shadowcarders.com/forums/carding-guides-tutorials.12/",
    "http://dumpsmabiyscb7ilk75z4cql7rxokimdrtbnfib2jaipu2dbylvsyyyd.darknyxproxy.com/",
    "http://plasticwrevezcuvtt6aj2bl3xx5q4x4wm2ptlf2jnjanpoulusjbnqd.darknyxproxy.com/us-cards.php",
    "http://ccpalymn3bi5uctrujnvrp5lf2nmyqf37oykanpcyts3oldt525mpnqd.darknyxproxy.com/",
    "http://imperiacfwnxzhkpyce2ifxwdf3zromko5wwrbaeldhtkvask3es7tyd.darknyxproxy.com/",
    "http://acteam2mf2ubo5hapf2dgsj4h7kjembfy4c6pmlkpueobucl2b3oyyqd.darknyxproxy.com/",
    "http://blackstgivygqcitkvmi6qwta3fpxhaakdjbhfe2dvfyahazm3etivqd.darknyxproxy.com/",
    "http://ccvendoqij2gteltjdd64usgdparw6kq2c2aqgmxjtmnoyd7vb45axad.darknyxproxy.com/",
    "https://ascarding.com/",
    "https://ascarding.com/forums/16/",
    "https://ascarding.com/forums/19/",
    "https://cardvilla.cc/carding-forum/?s=76185e0a2eb9e4631b4a6dfb26df4b34",
]

for website in websites:
    try:
        content = content_generation(website)
        content_list = [website,content,"Financial Crime"]
        df.loc[len(df)] = content_list
    except Exception as e:
        print(website,": ",e)


KeyboardInterrupt: 

In [22]:
df['Category'].value_counts()

Computers and Technology           303
Social Networking and Messaging    114
Business/Corporate                 106
E-Commerce                         101
News                                93
Law and Government                  83
Narcotics                           74
Financial Crime                     60
Adult                               43
Forums                              16
Name: Category, dtype: int64

In [14]:
df.to_csv("website_classification.csv",index=False)
# computers.to_csv("../Hierarchal model/Computer/computer_subcategory_classification.csv",index=False)